In [44]:
import MySQLdb
import random
import math

db = MySQLdb.connect(  # esto hace la conexion a la base de datos
    host="localhost",  # el puerto es 3308 porque es el puerto que
    user="root",  # le deje a mariaDB
    password="deadzamxd",
    database="PokeSearch",
    port=3308,
)
c = db.cursor()

In [2]:
TYPES = dict()
c.execute(f"select * from tipe;")
result = c.fetchall()
for tipe in result:
    TYPES[tipe[0]] = tipe[1]
relations = dict()
for t_id in TYPES:
    if t_id not in relations:
        relations[t_id] = dict()
    c.execute(
        f"select relation,id_type_2 from relation_type where id_type_1={t_id} and relation like '%from' ;"
    )
    temp = c.fetchall()
    for rt in temp:
        if rt[0] not in relations[t_id]:
            relations[t_id][rt[0]] = []
            relations[t_id][rt[0]].append(rt[1])
        else:
            relations[t_id][rt[0]].append(rt[1])


In [3]:
def damages(T: list):
    D = dict()
    for t in T:
        for rt in relations[t]:
            for ti in relations[t][rt]:
                if ti in D:
                    if rt == "double_damage_from":
                        D[ti] *= 2
                    elif rt == "half_damage_from":
                        D[ti] *= 0.5
                    else:
                        D[ti] *= 0
                else:
                    if rt == "double_damage_from":
                        D[ti] = 2
                    elif rt == "half_damage_from":
                        D[ti] = 0.5
                    else:
                        D[ti] = 0
    return D



In [4]:
class Pokemon:
    def __init__(self, name: str):
        c.execute(
            f"select id,atk,satk,def,sdef,total_stat,name,hp,spe from pokemon where name='{name}';"
        )
        data = c.fetchone()
        self.id = data[0]
        self.atk = data[1]
        self.defe = data[3]
        self.satk = data[2]
        self.sdef = data[4]
        self.total = data[5]
        self.name = data[6]
        self.hp = data[7]
        self.spe = data[8]
        c.execute(f"select id_type from pok_typ where id_pokemon={data[0]};")
        T = c.fetchall()
        self.types = []
        for t in T:
            self.types.append(t[0])
        self.damages = damages(self.types)

        self.moves = {"physical": dict(), "status": dict(), "special": dict()}
        self.all = []
        c.execute(
            f"select M.name,M.id_type,M.damage_class from move as M inner join pok_mov as PM on  PM.id_move=M.id and PM.id_pokemon={self.id};"
        )
        M = c.fetchall()
        for att in M:
            if att[1] not in self.moves[att[2]]:
                self.moves[att[2]][att[1]] = list()
            self.moves[att[2]][att[1]].append(att[0])
            self.all.append(att[0])

    def print(self):
        print(f"""id = {self.id}\natk = {self.atk}\ndefe = {self.defe}\nsatk = {self.satk}\nsdef = {self.sdef}\ntotal = {self.total}\nname = {self.name}""")
        for d in self.damages:
            print(TYPES[d],":",self.damages[d])
        for M in self.moves:
            print(M)
            for m in self.moves[M]:
                print(f"{TYPES[m]}:{self.moves[M][m]}")


In [119]:
class PokemonT:
    def __init__(self,poke: Pokemon,moves =[]):
        self.hp = math.floor(110+((2*poke.hp)+31+(252/4)))
        self.eva = 3
        self.pre = 3
        self.spe = poke.spe
        self.atk = poke.atk
        self.defe = poke.defe
        self.satk = poke.satk
        self.sdef = poke.sdef
        self.name = poke.name
        self.types = [TYPES[t] for t in poke.types]
        self.damages = {TYPES[k]:v for (k,v) in poke.damages.items() }
        for i in range(len(moves)):
            if moves[i] not in poke.all:
                moves[i] = random.choice(poke.all)
        moves.extend([random.choice(poke.all) for _ in range(4-len(moves))])
        self.moves = [moveT(m) for m in moves]
        

In [112]:
c.execute(f'select * from pokemon where name="charizard";')
data = c.fetchall()
data

((6,
  'charizard',
  905,
  17,
  78,
  84,
  78,
  109,
  85,
  100,
  534,
  'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/6.png'),)

In [39]:
class moveT:
    def __init__(self,name:str):
        c.execute(
            f"select * from move where name='{name}';"
        )
        data = c.fetchone()
        self.id = data[0]
        self.name = data[1]
        self.type = TYPES[data[2]]
        self.clss = data[3]
        self.power = data[4]
        self.pp = data[5]
        self.acc = data[6]
        self.prio = data[7]
        self.ail = data[8]
        self.ail_c = data[9]
        self.eff = data[10]

    def print(self):
        print(f"id = {self.id}\nname = {self.name}\ntype = {self.type}\nclss = {self.clss}\npower = {self.power}\npp = {self.pp}\nacc = {self.acc}\nprio = {self.prio}\nail = {self.ail}\nail_c = {self.ail_c}\neffect = {self.eff}")

In [15]:
m = moveT("mega-kick")
m.print()

id = 25
name = mega-kick
type = normal
clss = physical
power = 120
pp = 5
acc = 75
prio = 0
ail = none
ail_c = None
effect = Inflicts regular damage.


In [33]:
p1 = Pokemon("pikachu")
p1.print()
p2 = Pokemon("charizard")
#p2.print()


id = 25
atk = 55
defe = 40
satk = 50
sdef = 50
total = 320
name = pikachu
ground : 2
flying : 0.5
steel : 0.5
electric : 0.5
physical
normal:['mega-punch', 'pay-day', 'slam', 'mega-kick', 'headbutt', 'body-slam', 'take-down', 'double-edge', 'strength', 'quick-attack', 'rage', 'bide', 'skull-bash', 'return', 'frustration', 'facade', 'endeavor', 'secret-power', 'covet', 'natural-gift', 'feint']
electric:['thunder-punch', 'spark', 'volt-tackle', 'wild-charge', 'nuzzle']
fighting:['double-kick', 'submission', 'counter', 'seismic-toss', 'reversal', 'dynamic-punch', 'rock-smash', 'focus-punch', 'brick-break', 'upper-hand']
ground:['dig']
dark:['thief', 'knock-off', 'fling']
rock:['rollout']
steel:['iron-tail']
fairy:['play-rough']
grass:['trailblaze']
status
normal:['tail-whip', 'growl', 'mimic', 'double-team', 'defense-curl', 'flash', 'substitute', 'protect', 'endure', 'swagger', 'attract', 'sleep-talk', 'encore', 'helping-hand', 'captivate', 'play-nice', 'confide', 'laser-focus']
electric:

In [121]:
test1 = PokemonT(poke=p1,moves=['mega-kick', 'laser-focus', 'zap-cannon', 'uproar'])
test2 = PokemonT(poke=p2)
test2.hp

360

In [41]:
test1.moves[2].type


'electric'

In [98]:
def attack(user:PokemonT,move:moveT, target:PokemonT):
    if move.clss == "special": 
        Ataque = user.satk
        Defensa = target.sdef
    elif move.clss == "physical":
        Ataque = user.atk
        Defensa = target.defe
    if move.type in user.types: STAB = 1.5
    else: STAB = 1
    if move.type in target.damages:
        effect = target.damages[move.type]
    else:
        effect = 1

    rnd = (random.randint(217,255)*100)/255
    #damage = ((((((100 * 2 / 5) + 2) * move.power * Ataque / 50) / Defensa) * 1 + 2) * 1 * 1 * rnd / 100) * STAB * effect
    damage = math.floor(((((((100 * 2 / 5) + 2) * move.power * Ataque / 50) / Defensa) + 2)  * rnd / 100) * STAB * effect)
    print(damage)
    #acc = (move.acc/100)*((user.pre/3)/(target.eva/3))
    #print(acc)

In [107]:
attack(test1,test1.moves[2],test2)

178


Daño (PS) = [{([{(Nv. * 2 / 5) + 2} * Poder * Ataque / 50] / Defensa) * 1 + 2} * CH * 1 * Rnd / 100] * STAB * Efec.Tipo#1 * Efec.Tipo#2 * 1

Donde:

Nv.=El Nivel del Pokémon usuario.
Poder=Poder del Movimiento que el Pokémon usuario utiliza.
Ataque=Ataque o Ataque Especial del Pokémon usuario; depende de la Categoría del Movimiento: Si es Físico se utiliza el Ataque, si es Especial se utiliza el Ataque Especial.
Defensa=Defensa o Defensa Especial del Pokémon rival; depende de la Categoría del Movimiento: Si es Físico se utiliza la Defensa, si es Especial se utiliza la Defensa Especial.
1°Mod.=Primer modificador de la fórmula, para ver como influye mire la sección 2.4.
CH (Critical Hit) =Modificador de la fórmula equivalente a.....
» 2 si el Movimiento es Golpe Crítico.
» 1 para cualquier otro caso.
2°Mod.= Segundo modificador de la fórmula, para ver como influye mire la sección 2.5.
Rnd (Random)=Número entero de valor aleatorio que va desde 85 hasta 100. Rnd = (R * 100 / 255). Donde R es un número entero de valor aleatorio que va desde 217 hasta 255 con uniforme probabilidad de aparición.
STAB (Same Type Attack Bonus)=Modificador de la fórmula equivalente a.....
» 1.5 si el Tipo del Movimiento coincide con alguno de los Tipos del Pokémon usuario.
» 1 para cualquier otro caso.
Efec.Tipo#1 = Modificador de la fórmula equivalente a.....
» 2 si el Movimiento es muy efectivo con respecto al 1° Tipo del Pokémon rival.
» 0.5 si el Movimiento no es muy efectivo con respecto al 1° Tipo del Pokémon rival.
» 0 si el Movimiento no afecta al 1° Tipo del Pokémon rival.
» 1 para cualquier otro caso.
Ver el cuadro de efectividades.
Efec.Tipo#2=Modificador de la fórmula equivalente a.....
» 2 si el Movimiento es muy efectivo con respecto al 2° Tipo del Pokémon rival.
» 0.5 si el Movimiento no es muy efectivo con respecto al 2° Tipo del Pokémon rival
» 0 si el Movimiento no afecta al 2° Tipo del Pokémon rival.
» 1 si el rival solo tiene un Tipo o para cualquier otro caso.
Ver el cuadro de efectividades.
3°Mod.= Tercer modificador de la fórmula, para ver como influye mire la sección 2.6.